# 🧮 LSTM Model Eğitimi - Google Colab

Bu notebook Modern LSTM modelini JetX verileri üzerinde eğitir.

## 📋 LSTM Özellikleri:
- **Sequential Learning**: Zaman serisi sequence modeling
- **Multi-output**: Value, probability, confidence prediction
- **Attention Enhanced**: Multi-head attention mechanism
- **JetX Optimized**: Enhanced feature engineering
- **GPU Accelerated**: TensorFlow CUDA desteği

---

**⚠️ Önce `colab_setup.ipynb` notebook'unu çalıştırın!**

In [ ]:
# 🔧 Kurulum ve Import'lar
import sys
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import time
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Proje path'i ekle
sys.path.insert(0, '/content/predictor_1/src')

# Model import'ları
from models.sequential.lstm_model import ModernLSTMModel
from data_processing.loader import load_data_from_sqlite
from data_processing.splitter import create_sequences

# GPU kontrolü
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_virtual_device_configuration(
        physical_devices[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)]
    )
    print(f"🎯 GPU Device: {physical_devices[0]}")
    print(f"📱 GPU Available: {tf.config.list_physical_devices('GPU')}")
else:
    print("⚠️ GPU not found. Using CPU.")

print(f"📊 TensorFlow Version: {tf.__version__}")

# Matplotlib için stil
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ LSTM kurulumu tamamlandı!")

In [ ]:
# 📊 Veri Yükleme
print("📊 JETX VERİSİ YÜKLEME")
print("=" * 50)

# Colab'dan veritabanını yükle
db_path = "/content/colab_jetx_data.db"
df = load_data_from_sqlite(db_path)

# Veriyi liste formatına çevir
jetx_data = [row[1] for row in df.data]  # value column
print(f"✅ {len(jetx_data)} JetX verisi yüklendi")

# Veri analizi
jetx_array = np.array(jetx_data)
print(f"📈 İstatistikler:")
print(f"   - Min: {jetx_array.min():.2f}")
print(f"   - Max: {jetx_array.max():.2f}")
print(f"   - Mean: {jetx_array.mean():.2f}")
print(f"   - Std: {jetx_array.std():.2f}")
print(f"   - Crash rate (<1.5): {(jetx_array < 1.5).mean()*100:.1f}%")

# LSTM için veri görselleştirmesi
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Sequential pattern analysis
axes[0,0].plot(jetx_data[:200], 'b-', alpha=0.7, linewidth=1.5)
axes[0,0].set_title('Sequential Pattern Analysis (İlk 200 değer)')
axes[0,0].set_xlabel('Time Steps')
axes[0,0].set_ylabel('JetX Value')
axes[0,0].axhline(y=1.5, color='r', linestyle='--', alpha=0.7, label='Threshold')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Sequence length analysis
sequence_lengths = [10, 20, 50, 100, 150, 200]
correlations = []
for seq_len in sequence_lengths:
    if len(jetx_data) > seq_len:
        # Auto-correlation for different sequence lengths
        corr = np.corrcoef(jetx_data[:-seq_len], jetx_data[seq_len:])[0,1]
        correlations.append(corr)
    else:
        correlations.append(0)

axes[0,1].plot(sequence_lengths, correlations, 'ro-', linewidth=2, markersize=8)
axes[0,1].set_title('Auto-correlation vs Sequence Length')
axes[0,1].set_xlabel('Sequence Length')
axes[0,1].set_ylabel('Auto-correlation')
axes[0,1].grid(True, alpha=0.3)

# Distribution analysis
axes[1,0].hist(jetx_data, bins=50, alpha=0.7, edgecolor='black', color='skyblue')
axes[1,0].axvline(x=1.5, color='r', linestyle='--', alpha=0.7, linewidth=2, label='Threshold')
axes[1,0].axvline(x=np.mean(jetx_data), color='orange', linestyle='-', alpha=0.7, linewidth=2, label='Mean')
axes[1,0].set_title('Value Distribution')
axes[1,0].set_xlabel('JetX Value')
axes[1,0].set_ylabel('Frequency')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# Trend analysis
window = 50
trends = []
for i in range(window, len(jetx_data)):
    trend = np.polyfit(range(window), jetx_data[i-window:i], 1)[0]
    trends.append(trend)

axes[1,1].plot(trends[:300], alpha=0.7, linewidth=1.5)
axes[1,1].set_title('Local Trend Analysis')
axes[1,1].set_xlabel('Time Steps')
axes[1,1].set_ylabel('Trend Slope')
axes[1,1].axhline(y=0, color='r', linestyle='--', alpha=0.7)
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("🎯 LSTM veri analizi tamamlandı!")

In [ ]:
# 🔧 LSTM İçin Veri Hazırlama
print("🔧 LSTM İÇİN VERİ HAZIRLAMA")
print("=" * 50)

# LSTM parametreleri
SEQUENCE_LENGTH = 100  # LSTM için optimal sequence length
TRAIN_SPLIT = 0.8
VALIDATION_SPLIT = 0.1
BATCH_SIZE = 32

# Veriyi train/val/test olarak böl
n_train = int(len(jetx_data) * TRAIN_SPLIT)
n_val = int(len(jetx_data) * VALIDATION_SPLIT)

train_data = jetx_data[:n_train]
val_data = jetx_data[n_train:n_train + n_val]
test_data = jetx_data[n_train + n_val:]

print(f"📊 Veri bölümü:")
print(f"   - Train: {len(train_data)} samples")
print(f"   - Validation: {len(val_data)} samples")
print(f"   - Test: {len(test_data)} samples")

# Normalizasyon
train_mean = np.mean(train_data)
train_std = np.std(train_data)

print(f"📏 Normalizasyon parametreleri:")
print(f"   - Mean: {train_mean:.3f}")
print(f"   - Std: {train_std:.3f}")

# Normalize data (optional for LSTM)
def normalize_data(data, mean, std):
    return (np.array(data) - mean) / std

def denormalize_data(data, mean, std):
    return (data * std) + mean

# Sequences hazırlama
def prepare_lstm_sequences(data, sequence_length):
    """LSTM için sequence hazırlama"""
    X, y = [], []
    
    for i in range(len(data) - sequence_length):
        seq = data[i:i + sequence_length]
        target = data[i + sequence_length]
        
        X.append(seq)
        y.append(target)
    
    return np.array(X), np.array(y)

# Training sequences
X_train, y_train = prepare_lstm_sequences(train_data, SEQUENCE_LENGTH)
X_val, y_val = prepare_lstm_sequences(val_data, SEQUENCE_LENGTH)
X_test, y_test = prepare_lstm_sequences(test_data, SEQUENCE_LENGTH)

print(f"📊 Sequence boyutları:")
print(f"   - X_train: {X_train.shape}")
print(f"   - y_train: {y_train.shape}")
print(f"   - X_val: {X_val.shape}")
print(f"   - y_val: {y_val.shape}")
print(f"   - X_test: {X_test.shape}")
print(f"   - y_test: {y_test.shape}")

# Reshape for LSTM (samples, timesteps, features)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

print(f"📊 Reshaped boyutları:")
print(f"   - X_train: {X_train.shape}")
print(f"   - X_val: {X_val.shape}")
print(f"   - X_test: {X_test.shape}")

print("✅ LSTM veri hazırlama tamamlandı!")

In [ ]:
# 🧮 LSTM Model Oluşturma
print("🧮 LSTM MODEL OLUŞTURMA")
print("=" * 50)

# LSTM model parametreleri
model_params = {
    'seq_length': SEQUENCE_LENGTH,
    'n_features': 1,
    'threshold': 1.5,
    'multi_output': True,
    'use_attention': True,
    'use_residual': True
}

print(f"🔧 LSTM model parametreleri:")
for key, value in model_params.items():
    print(f"   - {key}: {value}")

# LSTM model oluştur
lstm_model = ModernLSTMModel(**model_params)

# Model build et
model = lstm_model.build_model(
    lstm_units=128,
    dropout_rate=0.2,
    learning_rate=0.001,
    attention_heads=4,
    num_layers=2
)

print(f"📊 LSTM model özeti:")
model.summary()

# Model compile bilgisi
print(f"\n🔧 Model compile bilgisi:")
print(f"   - Optimizer: Adam")
print(f"   - Loss: Multi-output (MSE + Binary Crossentropy)")
print(f"   - Metrics: MAE, Accuracy")
print(f"   - GPU Support: {tf.config.list_physical_devices('GPU')}")

print("✅ LSTM model hazır!")

In [ ]:
# 🎯 Multi-output Targets Hazırlama
print("🎯 MULTI-OUTPUT TARGETS HAZIRLAMA")
print("=" * 50)

# Multi-output targets oluştur
def create_multi_targets(y_values, threshold=1.5):
    """Multi-output targets oluştur"""
    targets = {
        'value_output': y_values,
        'probability_output': (y_values >= threshold).astype(float),
        'confidence_output': np.ones_like(y_values),  # Başlangıçta 1.0
        'crash_risk_output': (y_values < threshold).astype(float)
    }
    return targets

# Multi-output targets
y_train_multi = create_multi_targets(y_train)
y_val_multi = create_multi_targets(y_val)
y_test_multi = create_multi_targets(y_test)

print(f"📊 Multi-output target boyutları:")
for key, value in y_train_multi.items():
    print(f"   - {key}: {value.shape}")

# Target istatistikleri
print(f"\n📈 Target istatistikleri:")
print(f"   - Value range: [{y_train.min():.2f}, {y_train.max():.2f}]")
print(f"   - Above threshold: {y_train_multi['probability_output'].mean()*100:.1f}%")
print(f"   - Crash risk: {y_train_multi['crash_risk_output'].mean()*100:.1f}%")

print("✅ Multi-output targets hazırlandı!")

In [ ]:
# 🚀 LSTM Model Eğitimi
print("🚀 LSTM MODEL EĞİTİMİ")
print("=" * 50)

# Eğitim parametreleri
EPOCHS = 50
VERBOSE = 1

print(f"🎯 LSTM eğitimi başlıyor...")
print(f"   - Epochs: {EPOCHS}")
print(f"   - Batch size: {BATCH_SIZE}")
print(f"   - Sequence length: {SEQUENCE_LENGTH}")
print(f"   - Multi-output: {model_params['multi_output']}")

# Eğitim başlangıç zamanı
start_time = time.time()

# LSTM model eğitimi
try:
    history = model.fit(
        X_train, y_train_multi,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_val, y_val_multi),
        verbose=VERBOSE
    )
    
    # Eğitim süresi
    training_time = time.time() - start_time
    print(f"\n⏱️ LSTM eğitim süresi: {training_time/60:.2f} dakika")
    
    # Eğitim sonuçları
    print(f"📊 LSTM final sonuçlar:")
    print(f"   - Train Loss: {history.history['loss'][-1]:.6f}")
    print(f"   - Val Loss: {history.history['val_loss'][-1]:.6f}")
    print(f"   - Value MAE: {history.history['value_output_mae'][-1]:.6f}")
    print(f"   - Probability Accuracy: {history.history['probability_output_accuracy'][-1]:.6f}")
    print(f"   - Epochs completed: {len(history.history['loss'])}")
    
    print("✅ LSTM model eğitimi tamamlandı!")
    
except Exception as e:
    print(f"❌ LSTM eğitim hatası: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# 📈 LSTM Eğitim Sonuçları Görselleştirme
if 'history' in locals():
    print("📈 LSTM EĞİTİM SONUÇLARI GÖRSELLEŞTİRME")
    print("=" * 50)
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # Overall loss
    axes[0,0].plot(history.history['loss'], 'b-', label='Training Loss', linewidth=2)
    axes[0,0].plot(history.history['val_loss'], 'r-', label='Validation Loss', linewidth=2)
    axes[0,0].set_title('LSTM: Overall Loss', fontsize=14)
    axes[0,0].set_xlabel('Epochs')
    axes[0,0].set_ylabel('Loss')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # Value prediction loss (MAE)
    axes[0,1].plot(history.history['value_output_mae'], 'b-', label='Training MAE', linewidth=2)
    axes[0,1].plot(history.history['val_value_output_mae'], 'r-', label='Validation MAE', linewidth=2)
    axes[0,1].set_title('LSTM: Value Prediction MAE', fontsize=14)
    axes[0,1].set_xlabel('Epochs')
    axes[0,1].set_ylabel('MAE')
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)
    
    # Probability accuracy
    axes[0,2].plot(history.history['probability_output_accuracy'], 'b-', label='Training Accuracy', linewidth=2)
    axes[0,2].plot(history.history['val_probability_output_accuracy'], 'r-', label='Validation Accuracy', linewidth=2)
    axes[0,2].set_title('LSTM: Probability Accuracy', fontsize=14)
    axes[0,2].set_xlabel('Epochs')
    axes[0,2].set_ylabel('Accuracy')
    axes[0,2].legend()
    axes[0,2].grid(True, alpha=0.3)
    
    # Crash risk accuracy
    axes[1,0].plot(history.history['crash_risk_output_accuracy'], 'b-', label='Training Accuracy', linewidth=2)
    axes[1,0].plot(history.history['val_crash_risk_output_accuracy'], 'r-', label='Validation Accuracy', linewidth=2)
    axes[1,0].set_title('LSTM: Crash Risk Accuracy', fontsize=14)
    axes[1,0].set_xlabel('Epochs')
    axes[1,0].set_ylabel('Accuracy')
    axes[1,0].legend()
    axes[1,0].grid(True, alpha=0.3)
    
    # Learning rate schedule (if available)
    epochs_range = range(1, len(history.history['loss']) + 1)
    axes[1,1].plot(epochs_range, history.history['loss'], 'g-', linewidth=2)
    axes[1,1].set_title('LSTM: Training Loss Trend', fontsize=14)
    axes[1,1].set_xlabel('Epochs')
    axes[1,1].set_ylabel('Loss')
    axes[1,1].grid(True, alpha=0.3)
    
    # Performance summary table
    performance_stats = {
        'Final Train Loss': f"{history.history['loss'][-1]:.6f}",
        'Final Val Loss': f"{history.history['val_loss'][-1]:.6f}",
        'Best Val Loss': f"{min(history.history['val_loss']):.6f}",
        'Final Value MAE': f"{history.history['value_output_mae'][-1]:.6f}",
        'Final Prob Accuracy': f"{history.history['probability_output_accuracy'][-1]:.4f}",
        'Training Time': f"{training_time/60:.2f} min"
    }
    
    axes[1,2].axis('off')
    table_data = [[k, v] for k, v in performance_stats.items()]
    table = axes[1,2].table(cellText=table_data, colLabels=['Metric', 'Value'], loc='center', cellLoc='left')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.5)
    
    # Tablo stilini güzelleştir
    for i in range(len(table_data) + 1):
        for j in range(2):
            cell = table[(i, j)]
            if i == 0:  # Header
                cell.set_facecolor('#40466e')
                cell.set_text_props(weight='bold', color='white')
            else:
                cell.set_facecolor('#f1f1f2')
    
    plt.suptitle('LSTM Model Training Results', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("✅ LSTM eğitim sonuçları görselleştirildi!")
else:
    print("⚠️ LSTM eğitim history bulunamadı.")

In [ ]:
# 🧪 LSTM Model Test ve Değerlendirme
print("🧪 LSTM MODEL TEST VE DEĞERLENDİRME")
print("=" * 50)

# Test tahminleri
print("🔍 LSTM test tahminleri yapılıyor...")
test_predictions = model.predict(X_test, verbose=0)

# Multi-output predictions
pred_values = test_predictions['value_output'].flatten()
pred_probabilities = test_predictions['probability_output'].flatten()
pred_confidences = test_predictions['confidence_output'].flatten()
pred_crash_risks = test_predictions['crash_risk_output'].flatten()

print(f"✅ LSTM {len(pred_values)} tahmin yapıldı")

# LSTM metrikleri hesapla
if len(pred_values) > 0:
    # Regression metrikleri
    mae = np.mean(np.abs(pred_values - y_test))
    mse = np.mean((pred_values - y_test) ** 2)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((pred_values - y_test) / y_test)) * 100
    
    # R² score
    ss_res = np.sum((y_test - pred_values) ** 2)
    ss_tot = np.sum((y_test - np.mean(y_test)) ** 2)
    r2 = 1 - (ss_res / ss_tot)
    
    # Classification metrikleri
    actual_binary = (y_test >= 1.5).astype(int)
    pred_binary = (pred_probabilities >= 0.5).astype(int)
    
    accuracy = np.mean(actual_binary == pred_binary)
    precision = np.sum((pred_binary == 1) & (actual_binary == 1)) / np.sum(pred_binary == 1) if np.sum(pred_binary == 1) > 0 else 0
    recall = np.sum((pred_binary == 1) & (actual_binary == 1)) / np.sum(actual_binary == 1) if np.sum(actual_binary == 1) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # Crash risk metrikleri
    crash_actual = (y_test < 1.5).astype(int)
    crash_pred = (pred_crash_risks >= 0.5).astype(int)
    crash_accuracy = np.mean(crash_actual == crash_pred)
    
    print(f"📊 LSTM TEST SONUÇLARI:")
    print(f"   Regression Metrics:")
    print(f"   - MAE: {mae:.4f}")
    print(f"   - RMSE: {rmse:.4f}")
    print(f"   - MAPE: {mape:.2f}%")
    print(f"   - R² Score: {r2:.4f}")
    print(f"   Classification Metrics:")
    print(f"   - Threshold Accuracy: {accuracy:.4f}")
    print(f"   - Precision: {precision:.4f}")
    print(f"   - Recall: {recall:.4f}")
    print(f"   - F1-Score: {f1:.4f}")
    print(f"   - Crash Risk Accuracy: {crash_accuracy:.4f}")
    print(f"   Confidence Metrics:")
    print(f"   - Mean Confidence: {np.mean(pred_confidences):.4f}")
    print(f"   - Confidence Std: {np.std(pred_confidences):.4f}")

print("✅ LSTM model değerlendirmesi tamamlandı!")

In [ ]:
# 📊 LSTM Tahmin Sonuçları Görselleştirme
if 'pred_values' in locals() and len(pred_values) > 0:
    print("📊 LSTM TAHMİN SONUÇLARI GÖRSELLEŞTİRME")
    print("=" * 50)
    
    fig, axes = plt.subplots(3, 2, figsize=(15, 18))
    
    # Actual vs Predicted values
    axes[0,0].scatter(y_test, pred_values, alpha=0.6, s=30, c='blue')
    axes[0,0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', alpha=0.8, linewidth=2)
    axes[0,0].set_xlabel('Actual Values')
    axes[0,0].set_ylabel('Predicted Values')
    axes[0,0].set_title('LSTM: Actual vs Predicted Values')
    axes[0,0].grid(True, alpha=0.3)
    
    # Time series comparison
    n_show = min(100, len(pred_values))
    x_axis = range(n_show)
    axes[0,1].plot(x_axis, y_test[:n_show], 'b-', label='Actual', alpha=0.8, linewidth=2)
    axes[0,1].plot(x_axis, pred_values[:n_show], 'r-', label='LSTM Predicted', alpha=0.8, linewidth=2)
    axes[0,1].axhline(y=1.5, color='g', linestyle='--', alpha=0.6, label='Threshold', linewidth=2)
    axes[0,1].set_xlabel('Time Steps')
    axes[0,1].set_ylabel('JetX Value')
    axes[0,1].set_title(f'LSTM: Time Series Comparison (İlk {n_show} tahmin)')
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)
    
    # Probability predictions
    axes[1,0].scatter(y_test, pred_probabilities, alpha=0.6, s=30, c='purple')
    axes[1,0].axhline(y=0.5, color='r', linestyle='--', alpha=0.8, label='Decision Threshold')
    axes[1,0].axvline(x=1.5, color='g', linestyle='--', alpha=0.8, label='Value Threshold')
    axes[1,0].set_xlabel('Actual Values')
    axes[1,0].set_ylabel('Predicted Probability')
    axes[1,0].set_title('LSTM: Probability Predictions')
    axes[1,0].legend()
    axes[1,0].grid(True, alpha=0.3)
    
    # Error analysis
    errors = pred_values - y_test
    axes[1,1].hist(errors, bins=30, alpha=0.7, edgecolor='black', color='skyblue')
    axes[1,1].axvline(x=0, color='r', linestyle='--', alpha=0.8, linewidth=2)
    axes[1,1].axvline(x=np.mean(errors), color='orange', linestyle='-', alpha=0.8, linewidth=2, label=f'Mean: {np.mean(errors):.3f}')
    axes[1,1].set_xlabel('Prediction Error')
    axes[1,1].set_ylabel('Frequency')
    axes[1,1].set_title('LSTM: Error Distribution')
    axes[1,1].legend()
    axes[1,1].grid(True, alpha=0.3)
    
    # Confidence vs Error
    axes[2,0].scatter(pred_confidences, np.abs(errors), alpha=0.6, s=30, c='green')
    axes[2,0].set_xlabel('Confidence')
    axes[2,0].set_ylabel('Absolute Error')
    axes[2,0].set_title('LSTM: Confidence vs Absolute Error')
    axes[2,0].grid(True, alpha=0.3)
    
    # Performance metrics radar
    metrics_names = ['MAE', 'RMSE', 'R²', 'Accuracy', 'Precision', 'Recall', 'F1-Score']
    # Normalize metrics to 0-1 scale for radar chart
    metrics_values = [
        1 - (mae / np.max(np.abs(y_test))),  # MAE (inverted)
        1 - (rmse / np.max(np.abs(y_test))),  # RMSE (inverted)
        max(0, r2),  # R²
        accuracy,  # Accuracy
        precision,  # Precision
        recall,  # Recall
        f1  # F1-Score
    ]
    
    bars = axes[2,1].bar(metrics_names, metrics_values, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7', '#DDA0DD', '#FFB6C1'], alpha=0.8)
    axes[2,1].set_ylabel('Score')
    axes[2,1].set_title('LSTM: Performance Metrics')
    axes[2,1].tick_params(axis='x', rotation=45)
    axes[2,1].set_ylim(0, 1)
    
    # Değerleri barların üstüne yaz
    for bar, value in zip(bars, metrics_values):
        height = bar.get_height()
        axes[2,1].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                      f'{value:.3f}', ha='center', va='bottom', fontsize=9)
    
    plt.suptitle('LSTM Model Prediction Analysis', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("✅ LSTM tahmin sonuçları görselleştirildi!")
else:
    print("⚠️ LSTM tahmin sonuçları bulunamadı.")

In [ ]:
# 💾 LSTM Model Kaydetme
print("💾 LSTM MODEL KAYDETME")
print("=" * 50)

# Model kaydetme dizini
model_dir = "/content/trained_models/"
os.makedirs(model_dir, exist_ok=True)

# Model dosya adı
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_filename = f"lstm_model_{timestamp}.h5"
model_path = os.path.join(model_dir, model_filename)

# LSTM model kaydet
try:
    model.save(model_path)
    print(f"✅ LSTM model kaydedildi: {model_path}")
    
    # Model bilgilerini kaydet
    model_info = {
        'model_type': 'Modern LSTM with Attention',
        'timestamp': timestamp,
        'parameters': model_params,
        'training_params': {
            'epochs': EPOCHS,
            'batch_size': BATCH_SIZE,
            'sequence_length': SEQUENCE_LENGTH
        },
        'training_time': f"{training_time/60:.2f} minutes" if 'training_time' in locals() else 'Unknown',
        'final_losses': {
            'train': history.history['loss'][-1] if 'history' in locals() else 'Unknown',
            'val': history.history['val_loss'][-1] if 'history' in locals() else 'Unknown'
        },
        'test_metrics': {
            'mae': mae if 'mae' in locals() else 'Unknown',
            'rmse': rmse if 'rmse' in locals() else 'Unknown',
            'mape': mape if 'mape' in locals() else 'Unknown',
            'r2': r2 if 'r2' in locals() else 'Unknown',
            'accuracy': accuracy if 'accuracy' in locals() else 'Unknown',
            'precision': precision if 'precision' in locals() else 'Unknown',
            'recall': recall if 'recall' in locals() else 'Unknown',
            'f1_score': f1 if 'f1' in locals() else 'Unknown',
            'crash_accuracy': crash_accuracy if 'crash_accuracy' in locals() else 'Unknown'
        },
        'data_info': {
            'train_size': len(X_train),
            'val_size': len(X_val),
            'test_size': len(X_test),
            'sequence_length': SEQUENCE_LENGTH,
            'features': 1
        },
        'model_architecture': {
            'multi_output': model_params['multi_output'],
            'use_attention': model_params['use_attention'],
            'use_residual': model_params['use_residual'],
            'total_params': model.count_params()
        }
    }
    
    # Model bilgilerini JSON olarak kaydet
    import json
    info_filename = f"lstm_info_{timestamp}.json"
    info_path = os.path.join(model_dir, info_filename)
    
    with open(info_path, 'w') as f:
        json.dump(model_info, f, indent=2)
    
    print(f"✅ LSTM model bilgileri kaydedildi: {info_path}")
    
    # Google Drive'a kopyala (optional)
    print("\n💡 Google Drive'a kaydetmek için:")
    print(f"   !cp {model_path} /content/drive/MyDrive/")
    print(f"   !cp {info_path} /content/drive/MyDrive/")
    
    # Model performans özeti
    print("\n📊 LSTM MODEL PERFORMANS ÖZETİ:")
    print(f"   🎯 MAE: {mae:.4f}" if 'mae' in locals() else "   🎯 MAE: Unknown")
    print(f"   🎯 RMSE: {rmse:.4f}" if 'rmse' in locals() else "   🎯 RMSE: Unknown")
    print(f"   🎯 R² Score: {r2:.4f}" if 'r2' in locals() else "   🎯 R² Score: Unknown")
    print(f"   🎯 Accuracy: {accuracy:.4f}" if 'accuracy' in locals() else "   🎯 Accuracy: Unknown")
    print(f"   🎯 F1-Score: {f1:.4f}" if 'f1' in locals() else "   🎯 F1-Score: Unknown")
    print(f"   ⏱️ Training Time: {training_time/60:.2f} minutes" if 'training_time' in locals() else "   ⏱️ Training Time: Unknown")
    print(f"   🔢 Total Parameters: {model.count_params():,}")
    
except Exception as e:
    print(f"❌ LSTM model kaydetme hatası: {e}")
    import traceback
    traceback.print_exc()

print("\n🎉 LSTM model eğitimi tamamlandı!")

# 🎯 LSTM Eğitimi Özeti

## ✅ Tamamlanan İşlemler:
1. **Veri Hazırlama** - JetX verisi sequence format'a çevrildi
2. **Model Oluşturma** - Modern LSTM with Attention GPU'da oluşturuldu
3. **Multi-output Eğitim** - Value, probability, confidence, crash risk
4. **Değerlendirme** - Kapsamlı regression ve classification metrikleri
5. **Görselleştirme** - Detaylı analiz ve performans grafikleri
6. **Kaydetme** - TensorFlow model ve metadata kaydedildi

## 📊 LSTM Model Özellikleri:
- **Architecture**: Bidirectional LSTM + Multi-head Attention
- **Input**: Sequential JetX değerleri (sequence_length x 1)
- **Output**: 4 farklı çıktı (value, probability, confidence, crash_risk)
- **Optimization**: Adam optimizer with learning rate scheduling
- **Device**: GPU accelerated with TensorFlow

## 🚀 LSTM Avantajları:
- **Sequential Memory**: Uzun dönem bağımlılıkları hatırlama
- **Multi-output**: Tek modelde çoklu tahmin
- **Attention**: Önemli time step'lere odaklanma
- **Residual Connections**: Gradient flow iyileştirmesi
- **Bidirectional**: Hem geçmiş hem gelecek bilgisi

## 📈 Model Performansı:
- **Value Prediction**: MAE, RMSE, R² metrikleri
- **Classification**: Accuracy, Precision, Recall, F1-Score
- **Crash Detection**: Özel crash risk accuracy
- **Confidence**: Tahmin güvenilirlik skoru

## 🔄 Sonraki Adımlar:
1. **Ensemble Model** - LSTM + N-Beats + TFT kombinasyonu
2. **Hyperparameter Tuning** - Grid search ve Bayesian optimization
3. **Feature Engineering** - Ek temporal features
4. **Model Comparison** - Tüm modellerin karşılaştırılması
5. **Production Deployment** - Real-time inference servisi

---

**🎉 LSTM eğitimi başarıyla tamamlandı!**